In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("spark://Jafas-MacBook-Pro.local:7077") \
    .appName('test') \
    .getOrCreate()

23/03/26 09:46:14 WARN Utils: Your hostname, Jafas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.130 instead (on interface en0)
23/03/26 09:46:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/26 09:46:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
df_green = spark.read.parquet('data/pq/green/*/*')

23/03/26 09:46:31 ERROR TransportRequestHandler: Error while invoking RpcHandler#receive() on RPC id 7113063248880279790
java.io.InvalidClassException: org.apache.spark.storage.BlockManagerMessages$RegisterBlockManager; local class incompatible: stream classdesc serialVersionUID = -5464070606352341135, local class serialVersionUID = -699032251763294541
	at java.base/java.io.ObjectStreamClass.initNonProxy(ObjectStreamClass.java:601)
	at java.base/java.io.ObjectInputStream.readNonProxyDesc(ObjectInputStream.java:2062)
	at java.base/java.io.ObjectInputStream.readClassDesc(ObjectInputStream.java:1909)
	at java.base/java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2235)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1744)
	at java.base/java.io.ObjectInputStream.readObject(ObjectInputStream.java:514)
	at java.base/java.io.ObjectInputStream.readObject(ObjectInputStream.java:472)
	at org.apache.spark.serializer.JavaDeserializationStream.readObject

23/03/26 09:46:41 ERROR TransportRequestHandler: Error while invoking RpcHandler#receive() on RPC id 8250489250451559191
java.io.InvalidClassException: org.apache.spark.storage.BlockManagerMessages$RegisterBlockManager; local class incompatible: stream classdesc serialVersionUID = -5464070606352341135, local class serialVersionUID = -699032251763294541
	at java.base/java.io.ObjectStreamClass.initNonProxy(ObjectStreamClass.java:601)
	at java.base/java.io.ObjectInputStream.readNonProxyDesc(ObjectInputStream.java:2062)
	at java.base/java.io.ObjectInputStream.readClassDesc(ObjectInputStream.java:1909)
	at java.base/java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2235)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1744)
	at java.base/java.io.ObjectInputStream.readObject(ObjectInputStream.java:514)
	at java.base/java.io.ObjectInputStream.readObject(ObjectInputStream.java:472)
	at org.apache.spark.serializer.JavaDeserializationStream.readObject

In [ ]:
# Rename some columns.
df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [ ]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [ ]:
# Rename some columns.
df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [ ]:
# Create the list of columns present in the two datasets
# while preserving the order of the columns of the green dataset.
common_colums = []

yellow_columns = set(df_yellow.columns)

for col in df_green.columns:
    if col in yellow_columns:
        common_colums.append(col)

In [ ]:
# Create a column `service_type` indicating where the data comes from.
from pyspark.sql import functions as F

In [ ]:
df_green_sel = df_green \
    .select(common_colums) \
    .withColumn('service_type', F.lit('green'))

In [ ]:
df_yellow_sel = df_yellow \
    .select(common_colums) \
    .withColumn('service_type', F.lit('yellow'))

In [ ]:
# Create a new DataFrame containing union of rows of green and yellow DataFrame.
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [ ]:
# To check if the combination of the two files worked, run the following code
df_trips_data.groupBy('service_type').count().show()

### Querying this data with SQL

In [ ]:
# First, let’s get all column names as a list.

df_trips_data.columns

In [ ]:
df_trips_data.registerTempTable('trips_data')

In [ ]:
spark.sql("""
SELECT
    service_type,
    count(1)
FROM
    trips_data
GROUP BY 
    service_type
""").show()

In [ ]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

In [ ]:
# The coalesce() is used to decrease the number of partitions in an efficient way.
df_result.coalesce(1).write.parquet('data/report/revenue/', mode='overwrite')